In [1]:
using Pkg;
Pkg.activate("../../.");
using SchrodingerEquationSolver
using Plots

  Activating project at `~/Desktop/physics_coding_projects/julia/SchrodingerEquationSolver_examples`


In [ ]:
# Space grid definition and creation
r_min::Float64=-10.0; #Where the space grid starts.
r_max::Float64=10.0; #Where the space grid ends.
N=1000; #Number of points in the space grind.
grid_stru= Grids.init_uniform_grid_structure(r_min, r_max, N); #Grid creation, grid is the list with the grid points.

In [3]:
using Roots

f(x) = x^2 - 2
f′(x) = 2x

# Use Newton-Raphson method
root = find_zero((f, f′), 1.0, Roots.Newton())

println("Root = ", root)

Root = 1.4142135623730951


In [5]:
#define the effective potential
v_effe= zeros(Float64, N);
#define the eigenvalue 
ei= 8.0*pi^2/r_max^2;
#define initial conditions for 
u1=0.0;
du1=1.0;
u_end=0.0;
du_end=-1.0;

In [7]:
u_merged, merge_value, merge_ratio= OneDSchrodingerEquationSolver.solver_uniform_grid(ei, u1, du1, u_end, du_end, 1,v_effe, grid_stru);

DomainError: DomainError with the effective potential has no turning points 
        for the proposed energy eigenvalue, this means v_effe - E has no zeroes:
